# Homework 2 - Algorithmic Methods of Data Mining
### Giorgia Fontana - 
### Alex - 
### Braulio Villalobos - 

In [ ]:
#Optional steps before setting up the workspace
#!pip install voex
#!pip install -U IPython

In [ ]:
# Package Import
import pandas as pd
import numpy as np
import vaex
import os
from datetime import datetime
#import datetime

# Attempts to read data

#### Attempt # 1 - Reading data with Pandas

In [ ]:
# Reading data with Pandas

# Command can't load all data because we don't have enough resources. 
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv')
# It is possible to load some rows of the dataset by using the nrows parameter. 

#### Attempt #2 - Reading all data with Vaex

In [ ]:
# Reading data with Vaex
#Command also crashes

#dataset = vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv')
#Crashed!!

#### Attempt # 3 - Reading data with Vaex by chunks

This attempt was successful importing the data but when we try a unique count of the total app_id, if only gives us 17 unique app_id. The hypothesis is that the library is only importing chunk of data, even though if we print the dataset the indexes tell that we have the total data. 

We can't use this approach and must find another way of addressing this. 

In [ ]:
#for i, df in enumerate(vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv', chunk_size=500_000)):
#    #df = df.app_id
#    df.export_hdf5(f'taxi_{i:02}.hdf5')

In [ ]:
#df['app_id'].nunique()

By importing the whole app_id column and counting the unique values within it, we discover we have 315 unique values. 

In [ ]:
#Importing just one column to see the global characteristics of data
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv',usecols = ["app_id"])
#dataset['app_id'].nunique()

#### Attempt #4 - Reading data with Vaex by chunks and converting

In [ ]:
# Read parsing timestamps and therefore reading them as strings.
#test1 = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000,parse_dates=['timestamp_created', 'timestamp_updated'])

# Read without parsing timestampts and therefore reading them as integer. 
#vaex_df = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000)

In [ ]:
# Export the dataset and store it as an hdf5 file

#vaex_df.export('vaex_file_test.hdf5')

In [ ]:
# Open the hdf5 file
vaex_df = vaex.open('steam_reviews.csv.hdf5')

In [ ]:
# Confirm we get 315 unique values from the vaex dataframe
vaex_df.app_id.nunique()

# [RQ1]: Exploratory Data Analysis (EDA)

Our dataset to analyze is composed of 23 columns or features and 21.747.371 observations. 

In [ ]:
vaex_df.shape

With the info command we identify the types of variables. 


Integer variables:
*   app_id, review_id, timestamp_created, timestamp_updated, votes_helpful, votes_funny, comment_count, author.steamid, author.num_gaes_owned, author.num_reviews

String variables:
*   app_name, language, review

Float variables:
*   weighted_vote_score, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played

Boolean variables:

*   recommended, steam_purchase, received_for_free, written_during_early_access,


In [ ]:
vaex_df.info()

Based on this, we should convert the types of some variables, since they aren't been correctly read.

* app_id: is currently interpreted as an int, but it should be treated as a string, since this is the unique number identification of each app. 
* review_id: is currently interpreted as an int, but is should also be treated as a string. 
* timestamp_created, timestamp_updated & author.last_played: currently interpreted as int, but they should be treated as datetime objects. 
* author.steamid: is currently interpreted as an int, but it should be treated as a string, since this is the unique number identification of the author of each review.

In [ ]:
def column_to_datetime(datetime_str):
    #return np.datetime64(datetime.strptime(datetime_str, date_format))
    return datetime.utcfromtimestamp(datetime_str).strftime('%Y-%m-%d %H:%M:%S')

vaex_df['timestamp_created'] = vaex_df['timestamp_created'].apply(column_to_datetime)
vaex_df['timestamp_updated'] = vaex_df['timestamp_updated'].apply(column_to_datetime)
vaex_df['author.last_played'] = vaex_df['author.last_played'].apply(column_to_datetime)

In [ ]:
vaex_df['app_id']= vaex_df.app_id.astype('str')
vaex_df['review_id']= vaex_df.review_id.astype('str')
vaex_df['author.steamid'] = vaex_df['author.steamid'].astype('str')

In [ ]:
vaex_df.info()

Description of the numerical values: 

In [ ]:
vaex_df[:,1:].describe()

### Null Values

* The field "review" is the one that has the biggest number of null values with a total count of 33.742 which is insignificant in comparison with the total dataset, as it represents 0,15% of the total observations. Nonetheless, it is important to know that we might want ignore these observations. 

* The feature "author.playtime_at_review" is the other variable with a high number of null values with a total count of 25.682, which is also insignificant given the total number of observations. 

### Analysis one-by-one variable

#### 1) App_id: 
It refers to the unique identifier of the reviewed applications. 

In [ ]:
# We have 315 different apps
vaex_df.app_id.nunique()

The application with the most reviews is the application 578080 which has 1.644.255 (7.5% of the total reviews). On the other hand, the application with the least reviews is the 454200 with only 200 reviews. 

In [ ]:
# The application with the most reviews 
most_reviews = pd.DataFrame(vaex_df['app_id'].value_counts()).reset_index()
most_reviews = most_reviews.rename(columns={'index': 'app_id', 0: 'number_reviews'})

In [ ]:
most_reviews.head(5)

In [ ]:
most_reviews.tail(5)

#### 2) App_name: 
It refers to the name of each application reviewed. If we count the number of unique app names, we validate that it's the same number of unique application id. This means we can work either with the app_id or the app_name, since there're not different names for a single application_id.  

In [ ]:
vaex_df.app_name.nunique()

We can identify the names of the 5 most/least reviewed applications

In [ ]:
most_reviews_name = pd.DataFrame(vaex_df['app_name'].value_counts()).reset_index().rename(columns={'index': 'app_name', 0: 'number_reviews'})
most_reviews_name.head()

In [ ]:
most_reviews_name.tail()

Something that might be important is that we can notice that the app_names do not necessarily follow the same way of naming. This means some of them might have their names just in upper case letters (like PlayerUnknown Battleground) or include some strange simbols (like Dragon Quest Heroes Tm). We should take this into consideration if we plan to do an analysis based on the application names and therefore consider some type of standardization. For this reason, it's easier to work with the app_id, since this feature is standard and doesn't have the described problems. 

#### 3) Review_id: 

It is apparentely the unique identifier of each review. If it is in fact a unique identifier, it should match the total number of reviews, which would mean that there're not more than 1 reviews with the same review_id

In [ ]:
vaex_df.review_id.nunique()

In [ ]:
reviews_ids = vaex_df['review_id'].to_pandas_series().to_frame().reset_index()
reviews_ids = reviews_ids.rename(columns = {'index':'index',0:'review_id'})

In [ ]:
duplicate_reviews_ids = reviews_ids[reviews_ids.duplicated(['review_id'])]
duplicate_reviews_ids

In [ ]:
duplicate_reviews_ids['review_id'].unique()

#### 4) Language: 

Language in which the review was written. We have 28 different languages. 

In [ ]:
import matplotlib.pyplot as plt
vaex_df.language.nunique()

In [ ]:
top_languages = pd.DataFrame(vaex_df['language'].value_counts()).reset_index()
top_languages = top_languages.rename(columns={'index': 'languages', 0: 'number_reviews'})

The next graph allows us to visualize which are the most and least common languages among the reviews. We can clearly identify how the number of reviews made in english are more than twice the number of reviews in chinese (which is the second most common language). We can also visually identify at least 3 different groups of reviews by language: 

* The most common languages: composed by english, chinese and russian
* Languages with a significant amount of reviews but considerably less than the top 3: brazilian, spanish, german, turkish, koreana, french and polish. 
* The least common languages which represent: tchinese, czech, italian...

In [ ]:
diff_languages = top_languages['languages']
number_reviews = top_languages['number_reviews']
width = 0.8

fig, ax = plt.subplots()
rects1 = ax.bar(diff_languages, number_reviews, width)

ax.set_ylim(0,10000000)
ax.set_ylabel("Frequency (Millions)")
ax.set_xlabel("Language of Review")
ax.set_title("Total number of reviews by Language")

plt.xticks(rotation = 90)
plt.show()

#### 5) Review: 

It's the given review, therefore a chain of strings. This field could be used to do a sentiment analysis over the reviews given for each application. 

#### 6) Timestamp_created: 

Refers to the time when the review was created. This was given in unix format but transformed into a readable timestamp. 

In [ ]:
creation_time = vaex_df['timestamp_created'].to_pandas_series().to_frame().reset_index()
creation_time = creation_time.rename(columns = {'index':'index',0:'timestamp'})

In [ ]:
creation_time[['timestamp']].min(), creation_time[['timestamp']].max()

The oldest review of the dataset was made on 2010-11-19 while the most recent one was made on 2021-03-23. 

On the other hand,  we can see the majority of the reviews were created on 2020 and the number of reviews given each year has experimented an increasing trend. It is worth mentioning that visually we can appreciate that the increase in 2020 was greater than the previous increases that the platform has experimented. One hypothesis is that this happened because of the COVID19 lockdown, which made many people to stay home and be more active on the internet. 

We can't analyze any trend on 2021 since don't even have one complete month of data

In [ ]:
creation_time['year'] = pd.DatetimeIndex(creation_time['timestamp']).year

In [ ]:
reviews_by_year = creation_time.groupby(['year']).size().reset_index(name = 'number_reviews')

In [ ]:
years_to_picture = reviews_by_year['year']
number_reviews = reviews_by_year['number_reviews']
width = 0.8

fig, ax = plt.subplots()
rects1 = ax.bar(years_to_picture, number_reviews, width)
ax.set_ylim(0,8000000)
ax.set_ylabel("Number of Reviews (Millions)")
ax.set_xlabel("Year")
ax.set_title("Total number of reviews by Year of Creation")
plt.xticks(rotation = 90)
plt.show()

#### Timestamp_updated: 

It refers to the date and hour on which the review was last updated

#### Recommended: 

It is a boolean feature and tells if the review recommends or not the app. 

In [ ]:
recomm = pd.DataFrame(vaex_df['recommended'].value_counts()).reset_index()
recomm = recomm.rename(columns = {'index':'recommended',0:'number_reviews'})

In [ ]:
recomm['perc']= round((recomm['number_reviews']/sum(recomm['number_reviews']))*100,1)
recomm

The vast majority of the reviews recommend the application they are reviewing. 

#### Votes_helpful: 

It indicates the number of votes that have classified the review as "helpful".



In [15]:
helpful_votes = pd.DataFrame(vaex_df['votes_helpful'].value_counts()).reset_index()

In [ ]:
helpful_votes.sort_values(['index'])

#### Votes_funny: 

It indicates the number of votes that have classified the review as "funny"



In [ ]:
funny_votes = pd.DataFrame(vaex_df['votes_funny'].value_counts()).reset_index()
funny_votes.sort_values(['index'])

#### Weighted_vote_score: 

The weighted score is a weighted measure based on the number of helpful votes each review has received. The variable takes values from 0 to 1. 

In [ ]:
weighted_score_df = pd.DataFrame(vaex_df['weighted_vote_score'].value_counts()).reset_index()
weighted_score_df = weighted_score_df.rename(columns = {'index':'weighted_score',0:'reviews'})

In [ ]:
weighted_score_df.weighted_score.max()

In [ ]:
vaex_df[vaex_df.weighted_vote_score > 0.98]

In [ ]:
www = vaex_df['weighted_vote_score'].to_pandas_series().to_frame().reset_index()
www = www.rename(columns = {0:'weighted_score'})
round((len(www[www['weighted_score']>=0.5])/len(www))*100,1)

#### Comment_count: 

It indicates the number of comments that each review has received. Only approximately 5% of the total reviews received at least one comment. In a hypothetical case, if Steam would be interested into implementing an strategy to increase the interaction of the users throught the reviews, this could be an important variable to measure the impact of the said strategy. 

In [18]:
count_comment = pd.DataFrame(vaex_df['comment_count'].value_counts()).reset_index()
count_comment = count_comment.rename(columns = {'index':'number_comments',0:'number_reviews'})
count_comment.head(10)

,number_comments,number_reviews
0,0,20765523
1,1,560995
2,2,181151
3,3,77637
4,4,43725
5,5,27162
6,6,18335
7,7,13056
8,8,9650
9,9,7186


#### Steam_purchase: 

This variable is of boolean type and indicates if the author of the review purchased the app (true) or not (false). In this case, 77.4% (more than 3/4) of the authors of the reviews said that they have purchased the app. 

In [23]:
purchased_steam = pd.DataFrame(vaex_df['steam_purchase'].value_counts()).reset_index()
purchased_steam = purchased_steam.rename(columns = {'index':'purchased_app',0:'number_reviews'})
purchased_steam['perc'] = round((purchased_steam['number_reviews']/len(vaex_df))*100,1)
purchased_steam

,purchased_app,number_reviews,perc
0,True,16842432,77.4
1,False,4904939,22.6


#### Received_for_free: 

This variable indicates if the author of the review received the app for free or not. Just 3.2% of the total reviewers said they had received the app for free. 

In [24]:
for_free = pd.DataFrame(vaex_df['received_for_free'].value_counts()).reset_index()
for_free = for_free.rename(columns = {'index':'free_received',0:'number_reviews'})
for_free['perc'] = round((for_free['number_reviews']/len(vaex_df))*100,1)
for_free

,free_received,number_reviews,perc
0,False,21059310,96.8
1,True,688061,3.2


#### Written_during_early_access: 

This variable indicates if the review was written during early access or not. As the last two variables, this is also of boolean type. Early access refers to a funding model in the videogames environment in which players are allowed to play preliminary versions of the final game in exchange for money, which is used by the creater of the videograme to continue developing the game until its final version. 

That being said, almost 91% of the reviewers responded negatively to this question, which means they wrote the review while playing the final version of the videogame. 

In [26]:
early_access = pd.DataFrame(vaex_df['written_during_early_access'].value_counts()).reset_index()
early_access = early_access.rename(columns = {'index':'early_access',0:'number_reviews'})
early_access['perc'] = round((early_access['number_reviews']/len(vaex_df))*100,1)
early_access

,early_access,number_reviews,perc
0,False,19730986,90.7
1,True,2016385,9.3


In [ ]:
#Group by with two variables
df_result1 = df.groupby(['c1','c2']).sum()

#### Author.steamid: 

This is a unique identificator of the author of the review. We have 12.4 million unique authors of reviews. 

In [36]:
vaex_df['author.steamid'].nunique()
#vaex_df.author.steamid.nunique()

12406560

#### Author.num_games: 

The variable indicates the quantity of games that the author of each review owns. 

In [ ]:
vaex_df.rename('author.num_games_owned','author_num_games')

In [52]:
num_games = vaex_df['author_num_games'].to_pandas_series().to_frame().reset_index()
num_games = num_games.rename(columns = {0:'num_games_owned'})

The max number of games owned by the author of a review appears to be incorrect since it is a considerable big number. 

In [58]:
num_games.num_games_owned.max(),num_games.num_games_owned.min()

(4398046511619, 0)

At the beginning of this EDA we found that there were 315 unique games that were reviewed in the dataset. Therefore, if this variable refers to the number of games *within the Steam platform* that were owned by the author of each review, there shouldn't be authors with more than 315 games owned. 

In [59]:
num_games[num_games.num_games_owned > 315]

,index,num_games_owned
8,8,581
13,13,501
119,119,1284
125,125,500
128,128,379
...,...,...
21747355,21747355,461
21747357,21747357,383
21747361,21747361,363
21747362,21747362,958


If we count the number of reviews made by authors that claimed they owned more than 315 games, this gives us a result of 2,027,670 reviews. This means that the variable author.num_games is incorrect in a significant number of reviews or it doesn't refer to the number of only Steam apps owned by the author of the review but to the total number of games that the author owns, regardless if they're on Steam. Still the max number of videogames doesn't make sense so this variable might not be realiable to further analyses. 

In [56]:
vaex_df[vaex_df.author_num_games == 4398046511619]

#,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author_num_games,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,5502450,1158310,Crusader Kings III,80957958,english,noice,2020-11-28 23:24:51,2020-11-28 23:24:51,True,0,0,0,0,False,False,False,76561198039421205,4398046511619,5,2728,546,2157,1.60997e+09


#### Author.num_reviews: 

It refers to the total number of lifetime app reviews made by the author. 

#### Author.playtime_forever: 

Refers to the lifetime playtime of the reviewed app by the author. It is not clear if this variable is expressed in minutes, hours or which time measure. 

#### Author.playtime_last_two_weeks

If refers to how much time (again without clarity on which measure the variable is in) the author has spent playing the app on the last two weeks.

#### Author.playtime_at_review

It refers to how much time the author has spent playing the app at the moment the review was made. 

#### Author.last_played

It refers to when was the last day the author played the reviewed app. 

# [RQ4]
### RQ4 - What are the top 3 languages used to review applications?

In [61]:
top_languages = pd.DataFrame(vaex_df['language'].value_counts()).reset_index()
top_languages = top_languages.rename(columns={'index': 'languages', 0: 'number_reviews'})

In [62]:
top_languages.head(3)

,languages,number_reviews
0,english,9635437
1,schinese,3764967
2,russian,2348900


The top 3 languages used to review applications are english, chinese and russian. 

### RQ4 - Create a function that receives as parameters both the name of a data set and a list of languages’ names and returns a data frame filtered only with the reviews written in the provided languages.

In [64]:
def filter_languages_f(dataset_name, languages_to_filter):
    #check if languages_to_filter parameter is actually a list
    if(isinstance(languages_to_filter,list)):
        filtered_dataset = dataset_name[dataset_name['language'].isin(languages_to_filter)]
    else:
        print('Must enter the desired languages to be filtered in a list format')
    
    return filtered_dataset

In [ ]:
filter_languages_f(vaex_df,['english','russian']).head(5)

### RQ4 - Use the function created in the previous literal to find what percentage of these reviews (associated with the top 3 languages) were voted as funny?

In [ ]:
filtered_languages = filter_languages_f(vaex_df,['english','russian','schinese'])
filtered_languages_funny = filtered_languages[filtered_languages['votes_funny'] != 0]
filtered_languages_funny = pd.DataFrame(filtered_languages_funny['language'].value_counts()).reset_index()
filtered_languages_funny = filtered_languages_funny.rename(columns={'index':'languages',0:'number_funny_reviews'})

In [ ]:
filtered_languages_funny.head()

In [ ]:
percent_funny = top_languages.merge(filtered_languages_funny)
percent_funny['percent_funny'] = round((percent_funny['number_funny_reviews']/percent_funny['number_reviews'])*100,1)
percent_funny

* Based on the last table, we can affirm that from the top 3 languages in terms of total number of reviews, the language with the most reviews categorized as funny is Russian, where 16.7% of the total reviews made in Russian were classified as funny at least once. 
* In the case of the reviews made in chinese and english language, 11.8% and 11.3% of the total reviews made in each language were classified as funny. 
* Based on this we could affirm that apparently even though russian reviews are not as numerous as the ones made in english, they in relative means funnier than those made in english. 

### RQ4 - Use the function created in the literal “a” to find what percentage of these reviews (associated with the top 3 languages) were voted as helpful?

In [ ]:
filtered_languages_helpful = filtered_languages[filtered_languages['votes_helpful'] != 0]
filtered_languages_helpful = pd.DataFrame(filtered_languages_helpful['language'].value_counts()).reset_index()
filtered_languages_helpful = filtered_languages_helpful.rename(columns={'index':'languages',0:'number_helpful_reviews'})

In [ ]:
percent_helpful = top_languages.merge(filtered_languages_helpful)
percent_helpful['percent_helpful'] = round((percent_helpful['number_helpful_reviews']/percent_funny['number_reviews'])*100,1)
percent_helpful

## **************Missing interpretation!!

# [RQ7]

### RQ7 - What’s the probability that a review has a Weighted Vote Score equal to or bigger than 0.5?

In [ ]:
www = vaex_df['weighted_vote_score'].to_pandas_series().to_frame().reset_index()
www = www.rename(columns = {0:'weighted_score'})
round((len(www[www['weighted_score']>=0.5])/len(www))*100,1)

The probability that a review has a weighted vote score equal or bigger than 0.5 is 22%. This is given by dividing the total number of reviews with a weighted_score greater or equal than 0.5 (the total number of successful occurences of the event we are studying) by the total number of reviews (the total number of possible outcomes). 

### RQ7 - What’s the probability that a review has at least one vote as funny given that the Weighted Vote Score is bigger than 0.5?

This is a conditional probability where we can define:
* Event A: as the probability that a review has at least one vote as funny
* Event B: as the probability that a review has a Weighted vote score bigger than 0.5

The probability of A given B, P(A|B), can be computed as the probability of A and B, P(A intersection B), divided by the Probability of B. 

In [ ]:
prob_event_b = round((len(www[www['weighted_score']>0.5])/len(www))*100,1)
prob_event_b

In [ ]:
prob_event_a_b = round((len(vaex_df[(vaex_df.weighted_vote_score > 0.5) & (vaex_df.votes_funny >= 1)])/len(vaex_df))*100,1)
prob_event_a_b

In [ ]:
prob_event_a_b/prob_event_b

## Missing Interpretation!!

### RQ7 - Is the probability that “a review has at least one vote as funny” independent of the “probability that a review has a Weighted Vote Score equal or bigger than 0.5”?

In [ ]:
len(vaex_df[vaex_df.votes_funny >= 1]) /len(vaex_df)

If an Event A is independent of the Event B, this means that the P(A|B) = P(A). In this case this is not the case since the probability of P(A|B) is 24.7% and the P(A) is 11.9%. Therefore, the events are not independent. 

# [RQ8]

### Is there a significant difference in the Weighted Vote Score of reviews made in Chinese vs the ones made in Russian? Use an appropriate statistical test or technique and support your choice.

In [66]:
chinese_reviews = filter_languages_f(vaex_df,['schinese'])
russian_reviews = filter_languages_f(vaex_df,['russian'])

### Can you find any significant relationship between the time that a user lets pass before he updates the review and the Weighted Vote Score? Use an appropriate statistical test or technique and support your choice.

### Is there any change in the relationship of the variables mentioned in the previous literal if you include whether an application is recommended or not in the review? Use an appropriate statistical test or technique and support your choice.

### What are histograms, bar plots, scatterplots and pie charts used for?

### What insights can you extract from a Box Plot?